# Assignment 7

### Part A

In [3]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [ ]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [39]:
def expected_return(P, rf, mkt_rp):
    betas = P['Beta'].tolist()
    vola = P['Volatility'].tolist()
    invest = P['Amount Invested'].tolist()
    final = []
    
    for i in range(len(betas)):
        weight = invest[i] / sum(invest)
        exp_return = (rf + betas[i]*mkt_rp)*weight
        final = final + [exp_return]
    return sum(final)

In [40]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })
expected_return(portfolio, 0.02, 0.05)

0.086

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [11]:
def volatility(P, cor):
    betas = P['Beta'].tolist()
    vola = P['Volatility'].tolist()
    invest = P['Amount Invested'].tolist()
    final = []
    second = 1
    
    for i in range(len(vola)):
        weight = invest[i] / sum(invest)
        first = ((weight)**2) * (vola[i])**2
        second = second * weight * vola[i]
        final = final + [first]
    second = second * 2 * cor
    return (sum(final) + second)**(1/2)

In [12]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })
volatility(portfolio, -0.2)

0.24083189157584595

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [13]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    mkt = vol / mkt_vol
    final = rf + (mkt * mkt_rp)
    return final


In [14]:
expected_return2(0.02, 0.05, 0.16, 0.2408)

0.09525

### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [18]:
def CAPM_beta(return_path, factors_path):
    df = pd.read_csv(return_path)
    df1 = pd.read_csv(factors_path)
    
    cusip = []
    exp_ret = []
    
    data = df.groupby('CUSIP')
    for col, row in data:
        cusip +=[col]
        ret = []
        for x in row['RET']:
            ret += [x]
        exp_ret += [sum(ret) / len(ret)]
    
    df['date'] = (df['year'] * 100) + df['month']
    df2 = pd.merge(df, df1, on='date')
    df2['a'] = df2['RET'] - df2['Mkt-RF']
    df2['b'] = df2['Mkt-RF'] - df2['RF']
    
    grouped = df2.groupby('CUSIP')
    betas = grouped.apply(lambda x: x['a'].cov(x['b'])) / grouped['b'].var()
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusip ,
        "EXP_RETURN" : exp_ret,
        "BETA" : betas,
    })
    
    return output

In [15]:
CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV")

CUSIP     object
RET      float64
year       int64
month      int64
dtype: object


CUSIP  EXP_RETURN      BETA
CUSIP                                   
00081T10  00081T10    0.016252 -0.970486
00108M10  00108M10    0.011101 -0.984346
00130H10  00130H10    0.016152 -0.982472
00163U10  00163U10    0.015538 -0.985973
00182C10  00182C10    0.014772 -0.982449
...            ...         ...       ...
9959910    9959910    0.009933 -0.984629
9972410    9972410    0.012866 -0.983943
9972510    9972510    0.017177 -0.980412
9984910    9984910    0.009006 -0.981763
9985510    9985510    0.015699 -0.980395

[3225 rows x 3 columns]

Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [31]:
def slope(data):
    y = data['EXP_RETURN']
    x = data['BETA']
    slope_ = np.polyfit(x, y, 1)[0]
    # This should return 0.007981588120535642... with the provided data.
    return slope_

In [32]:
slope(CAPM_beta("monthlycrsp_mod7.csv", "F-F_Research_Data_Factors.CSV"))

0.3852709007950812